# Face Detector and Gathering the data 

In [1]:

import cv2
import os

#Preparing Realtime Video Camera Window
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height

#Using 'haarcascade_frontalface_default.xml' 
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Entering The Person ID Starting From Number 1 ex: 1 = Mohamed , 2 = Yehya ,, etc
face_id = input('\nEnter User id ==> ')

print("\n [INFO] Initializing face capture. Look at the camera and wait ...")
# Initialize individual sampling face count
count = 0

#start detect your face and take 100 pictures
while(True):

    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1

        # Save the captured image into the datasets folder with the id and pic number
        cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)

    #You Can Press 'ESC' for exiting video
    k = cv2.waitKey(100) & 0xff 
    if k == 27:
        break
    elif count >= 100: #The Number of Pictures That the application takes for one user
         break

print("\n Exiting Program >> Collecting Dataset is done now we will train it")
cam.release()
cv2.destroyAllWindows()



Enter User id ==> 4

 [INFO] Initializing face capture. Look at the camera and wait ...

 Exiting Program >> Collecting Dataset is done now we will train it


# Face Training 

In [2]:

import numpy as np
from PIL import Image #pillow package
import os
import cv2
# Path for database
path = 'dataset'

#using haarcascade_frontalface_default.xml
detector= cv2.CascadeClassifier("haarcascade_frontalface_default.xml"); 
recognizer = cv2.face.LBPHFaceRecognizer_create()



# function to get the images and label data
def getImagesAndLabels(path):

    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []


    #seperate the name and id >> and save the faces x,y,w,h in the faceSamples array
    for imagePath in imagePaths:

        PIL_img = Image.open(imagePath).convert('L') 
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)

        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)

    return faceSamples,ids

print ("\n Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml')

# Print the numer of faces trained and end program
print("\n {0} faces trained. now we can recognize the trained faces".format(len(np.unique(ids))))



 Training faces. It will take a few seconds. Wait ...

 4 faces trained. now we can recognize the trained faces


# Face Recognition 

In [3]:

import cv2
import numpy as np
import os 

recognizer = cv2.face.LBPHFaceRecognizer_create()
#load trained model
recognizer.read('trainer/trainer.yml')   
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

#The id is the number of persons that we are try to recgonzie them
id = 4 #There is no ID 0 >> The IDs Starting from 1

#Names of the Persons, start from the second place
#the first place is 0 and there is no 0 ID
names = ['','Mohamed Nader','Yehya Sayed','Mohamed Hesam' , 'Omar Amgad' ]  

# Starting realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)

while True:

    ret, img =cam.read()

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )


    #trying to recognize the faces and comparing them with the dataset
    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
    
    cv2.imshow('camera',img) 

    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

print("\n Exiting Program")
cam.release()
cv2.destroyAllWindows()



 Exiting Program
